In [ ]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
import scipy as sp

#visualizing results
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context("poster")
sns.set_style("ticks")
#import yellowbrick as yb

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import silhouette_score
from sklearn.decomposition import KernelPCA
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import scipy.cluster.hierarchy as shc

from kmodes.kmodes import KModes

In [ ]:
data_path = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Clinical projects/PeskindTBI/final_data/first_visits_short.csv'

In [ ]:
#read in csv containing data from all surveys and all visitseqs
data = pd.read_csv(data_path, index_col=0)
data = pd.DataFrame(data = data)

print('Original data shape:\n', data.shape, '\n')
print(data.info())
data.head(1)

In [ ]:
#determined outliers for auditc and QBlstExp (outlier = >3 SD from mean) and remove
data = data[data["TBIID"] != 'C010']
data = data[data["TBIID"] != 'T080']

In [ ]:
#create variable lists depending on data type for graphing purposes

data_cont = ['ScreenAge', 'Education', 'servconn', 'cestotal',
       'MnthSncBlst', 'QKOIorA', 'QKOExpMil', 'QKOAllMil', 'QKOLife',
       'QBlstExp', 'QBEACRM', 'Q5plus2', 'QEDist_sum', 'QEDist_mean',
       'QEDist_min', 'DOPA', 'DA', 'DOPAC', 'NE', 'DHPG', 'da_dopa_ratio', 'dopac_da_ratio',
       'ne_dopa_ratio', 'dhpg_ne_ratio', 'HEIGHT', 'WEIGHT', 'BMI',
       'BPSYS', 'BPDIAS', 'HRATE', 'BGlucose', 'BNa', 'BUN', 'BCreat', 'UNa',
       'BK', 'TotalChol', 'LDL', 'HDL', 'Trig', 'CSFPROTEIN_x',
       'CSFGLUCOSE', 'CSFRBCS',  
       'CAPSTotal', 'PSQIc1', 'PSQIc2', 'PSQIc3', 'PSQIc4', 'PSQIc5',
       'PSQIc6', 'PSQIc7', 'PSQItot', 'PCLTot', 'PCL_reexp', 'PCL_avoid',
       'PCL_numb', 'PCL_hyper', 'PHQTot', 'PHQ_psych', 'PHQ_somatic',
       'NSITot', 'TBITot', 'NSI_vestibular', 'NSI_somatosensory',
       'NSI_cognitive', 'NSI_affective', 'NSI_ERP_affective',
       'NIS_ERP_vestsom', 'DvpHA', 'DvpHAAct', 'DvpHASlp',
       'DvpHAMd', 'DvpHAStr', 'DvpBP', 'DvpBPAct', 'DvpBPslp', 'DvpBPMd',
       'DvpBPStr', 'BISAtt', 'BISMtr', 'BISNonpl', 'BISTot',
             'PreSleep', 'PreCaff', 'PreETOH', 'PreNic', 'PreTHC']

data_cat = ['Status_x', 'VisitSeq', 'GType', 'Race', 'Hispanic', 'Handedness', 'Marital', 'APOEGen', 
           'HYPERTEN', 'HYPERCHO', 'DIABETES', 'B12DEF', 'THYROID',
           'MHxPain', 'MHxHA', 'MHxHtn', 'MHxCard', 'MHxGI', 'MHxNeuro', 'MHxLung', 'MHxApnea', 
            'SCPTSD', 'SCMDD', 'SCPD', 'SCGAD', 'SCNone', 'capsCrtA']

data_audit = ['auditc', 'AUDIT1', 'AUDIT2', 'AUDIT3']

In [ ]:
#viz for first visit seq

data_v1 = data[data['VisitSeq'] == 1]

for param in data_cat:
    print(param)

    data_int = (data_v1[data_v1[param] != 9].groupby('Group')[param].value_counts() /
                data_v1[data_v1[param] != 9].groupby('Group')[param].count()).reset_index(name='perc')
        
    try:
        g = sns.catplot(x=param, y='perc', kind='bar', data=data_int, hue='Group', ci=68, height=5, aspect=4)
        plt.show()
        
        print('\n')
        
    except:
        pass
    
for param in data_audit:
    print(param)

    data_int = (data.groupby('Group')[param].value_counts() /
                        data.groupby('Group')[param].count()).reset_index(name='perc')
        
    try:
        g = sns.catplot(x=param, y='perc', kind='bar', data=data_int, hue='Group', ci=68, height=5, aspect=4)
        plt.show()
        
        print('\n')
        
    except:
        pass
    
for param in data_cont:
    print(param)
    
    try:
        g = sns.catplot(x='Group', y=param, kind='bar', data=data_v1, ci=68, height=5, aspect=4)
        plt.show()
        
        print('\n')
        
    except:
        pass

### EDA and viz for followup visits and health outcomes

In [ ]:
#exclude 3rd visit seq
visits = [1,2]
data_v12 = data[data['VisitSeq'].isin(visits)]
#get TBIIDs of participants with a second visit and use to filter
TBIID_v2 = data_v12[data_v12['VisitSeq'] == 2]['TBIID'].values

#create new column for if came to second visitseq - want to see if params at visitseq 1 map on to whether they came back again
data_v12['visit2'] = [1 if TBIID in TBIID_v2 else 0 for TBIID in data_v12['TBIID']]

#create new df with only participants who came to both visits 1 and 2
data_v12_only = data_v12[data_v12['TBIID'].isin(TBIID_v2)]
print(data_v12_only.shape)
print(data_v12_only.groupby(['VisitSeq', 'Group'])['Group'].value_counts())
data_v12_only.head()

In [ ]:
#viz visit 1 and 2 (only include participants who came to both visits)

for param in data_cat:
    print(param)

    try:
        data_int = (data_v12_only[data_v12_only[param] != 9].groupby(['Group', 'VisitSeq'])[param].value_counts() /
                data_v12_only[data_v12_only[param] != 9].groupby(['Group', 'VisitSeq'])[param].count()).reset_index(name='perc')
        
        g = sns.catplot(x=param, y='perc', kind='bar', data=data_int, hue='Group', ci=68, height=5, aspect=4, col='VisitSeq', col_wrap=1)
        plt.show()
        
        print('\n')
        
    except:
        pass
    
for param in data_audit:
    print(param)

    data_int = (data_v12_only.groupby(['Group', 'VisitSeq'])[param].value_counts() /
                        data_v12_only.groupby(['Group', 'VisitSeq'])[param].count()).reset_index(name='perc')
        
    try:
        g = sns.catplot(x=param, y='perc', kind='bar', data=data_int, hue='Group', ci=68, height=5, aspect=4, col='VisitSeq', col_wrap=1)
        plt.show()
        
        print('\n')
        
    except:
        pass
    
for param in data_cont:
    print(param)
    
    try:
        g = sns.catplot(x='Group', y=param, kind='bar', hue='VisitSeq', data=data_v12_only, ci=68, height=5, aspect=4)
        plt.show()
        
        print('\n')
        
    except:
        pass